In [1]:
# This code reads in a lensing galaxy catalogue and appends to it 
# values of the mass over-density and tidal field tensor at the galaxy locations
# By Joachim Harnois-Deraps, 2021.

import numpy as np
import healpy as hp
#from scipy.interpolate import interp1d
import random
import input
from pathlib import Path


# The tomographic bin
tomobin = 1

print("**************************")
print("Processing tomo bin",tomobin)
print("**************************")

# Use the normal or smoothed tensor fields: '_V2' (0.1Mpc) or '_smooth' (0.5Mpc) or '_smooth_1Mpc' (1Mpc)
smooth_flag='_smooth'

# the number of galaxies per square arcminute to extract
gpam = 0.6

# The NSIDE value of the shear and weight maps
nside = 8192

#npix = 12*nside**2
#pix_size = np.sqrt(4.*np.pi*(180./np.pi*60)**2/npix)
#print('pixel size of shear maps is %f arcmin per side' % pix_size)

zfile=np.loadtxt("z2ts.txt",delimiter=',')
z_list=np.flip(zfile[1:58,0])
snaplist = np.flip(zfile[1:58,1].astype(int))
n_slices = np.size(z_list)


**************************
Processing tomo bin 1
**************************


In [3]:
#for plane in range(0,41):
for plane in range(0,n_slices):

    #d_smooth = '1' #'01', '05' or '1'
    d_smooth = '0.5' #'01', '05' or '1'
    
    print('starting analysis of plane %i' % plane)
            
    #V0
    #fname = '../../GalCat/StageIV_nz/GalCat_tomo'+np.str(tomobin)+'_plane'+np.str(plane)+'_'+np.str(gpam)+'GpAM_RA_Dec_g1_g2_w.asc'
    #fname = '../../GalCat/SRD-Y1/random_pos/GalCat_tomo'+np.str(tomobin)+'_plane'+np.str(plane)+'_'+np.str(gpam)+'GpAM_RA_Dec_g1_g2_w_z.asc'

    #V1
    #fname = '../../GalCat/StageIV_nz/V1/GalCat_tomo'+np.str(tomobin)+'_plane'+np.str(plane)+'_'+np.str(gpam)+'GpAM_RA_Dec_g1_g2_w_linear_bias.asc'
    #fname = '../../GalCat/SRD-Y1/lin_bias_pos/GalCat_tomo'+np.str(tomobin)+'_plane'+np.str(plane)+'_'+np.str(gpam)+'GpAM_RA_Dec_g1_g2_w_linear_bias_bta1.asc'
    #fname = '../../GalCat/SRD-Y1/lin_bias_pos/GalCat_tomo'+np.str(tomobin)+'_plane'+np.str(plane)+'_'+np.str(gpam)+'GpAM_RA_Dec_g1_g2_w_linear_bias_bta2.asc'
    fname = '../../GalCat/SRD-Y1/lin_bias_pos/GalCat_tomo'+str(tomobin)+'_plane'+str(plane)+'_'+str(gpam)+'GpAM_RA_Dec_g1_g2_w_linear_bias_smooth'+d_smooth+'_bta2.asc'
    # note : the *planes* named bta1 are now bta2.

    # check file exists:
    my_file = Path(fname)
    if my_file.is_file():
        # file exists
        print("Got file! Reading",fname)
        ra, dec, shear1, shear2, w, z = np.loadtxt(fname, unpack=True)
        
        #load mass maps and tidal tensor
        fname = "../../density/density_map_"+str(snaplist[plane])+"_dens_allsky.npy"
        #fname = input.deltaDir+"density_map_"+str(snaplist[plane])+"_dens_allsky_smooth.npy"

        print("opening ", fname) 
        hpmap_density = np.load(fname)
        print("Got density file!")
        
        fname = "../../tidalfield/tidal_field_map_"+str(snaplist[plane])+"_allsky"+smooth_flag+".npy"

        print("opening ", fname) 
        hpmap_tidal = np.load(fname)
        print("Got tidal file!")
        

        # Normalize the maps correctly:       
        mean_rho = (np.mean(hpmap_density)) # no factor of 8.0 since we have full sky data here
        hpmap_density /= mean_rho 
        hpmap_density -= 1
        
        Norm_empirical = 0.6525 
        
        hpmap_tidal *= Norm_empirical

        print("over density:",np.min(hpmap_density), np.max(hpmap_density), np.mean(hpmap_density))
        print("s11:",np.min(hpmap_tidal[:,0]), np.max(hpmap_tidal[:,0]), np.mean(hpmap_tidal[:,0]))
        print("s22:",np.min(hpmap_tidal[:,1]), np.max(hpmap_tidal[:,1]), np.mean(hpmap_tidal[:,1]))
        print("s12:",np.min(hpmap_tidal[:,2]), np.max(hpmap_tidal[:,2]), np.mean(hpmap_tidal[:,2]))


        
        # Interpolate maps at galaxy positions
        den_pix = hp.get_interp_val(hpmap_density, (90.-dec)*np.pi/180., ra*np.pi/180., nest=False )

        s11_pix = hp.get_interp_val(hpmap_tidal[:,0], (90.-dec)*np.pi/180., ra*np.pi/180., nest=False )
        s22_pix = hp.get_interp_val(hpmap_tidal[:,1], (90.-dec)*np.pi/180., ra*np.pi/180., nest=False )
        s12_pix = hp.get_interp_val(hpmap_tidal[:,2], (90.-dec)*np.pi/180., ra*np.pi/180., nest=False )



        # Save a shear catalogue in standard format to be read by Athena
        #outname = '../../GalCat/SRD-Y1/random_pos/GalCat_tomo'+str(tomobin)+'_plane'+str(plane)+'_'+str(gpam)+'GpAM_RA_Dec_g1_g2_w_z_den_Sij'+smooth_flag+'.asc'
        #outname = input.GalDir+'StageIV_nz/V0/GalCat_tomo'+str(tomobin)+'_plane'+str(plane)+'_'+str(gpam)+'GpAM_RA_Dec_g1_g2_w_den_Sij.asc'
        #outname = '../../GalCat/StageIV_nz/V1/GalCat_tomo'+str(tomobin)+'_plane'+str(plane)+'_'+str(gpam)+'GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias'+smooth_flag+'.asc'
        #outname = '../../GalCat/StageIV_nz/V1/GalCat_tomo'+str(tomobin)+'_plane'+str(plane)+'_'+str(gpam)+'GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias'+smooth_flag+'_bta2.asc'
        #outname = '../../GalCat/SRD-Y1/lin_bias_pos/GalCat_tomo'+str(tomobin)+'_plane'+str(plane)+'_'+str(gpam)+'GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias'+smooth_flag+'_bta1.asc'
        #outname = '../../GalCat/SRD-Y1/lin_bias_pos/GalCat_tomo'+str(tomobin)+'_plane'+str(plane)+'_'+str(gpam)+'GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias'+smooth_flag+'_bta2.asc'
        #outname = '../../GalCat/SRD-Y1/lin_bias_pos/GalCat_tomo'+str(tomobin)+'_plane'+str(plane)+'_'+str(gpam)+'GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth'+d_smooth+'_'+smooth_flag+'_bta1.asc'
        outname = '../../GalCat/SRD-Y1/lin_bias_pos/GalCat_tomo'+str(tomobin)+'_plane'+str(plane)+'_'+str(gpam)+'GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth'+d_smooth+'_'+smooth_flag+'_bta2_new.asc'


        print("Wrinting",outname)

        # given ra=np.degrees(phi), e2 DOES NOT NEED FLIPPING.
        #np.savetxt(outname, np.c_[ra_rand, dec_rand, shear1_pix_front*dz1 + shear1_pix_back*dz2, shear2_pix_front*dz1 + shear2_pix_back*dz2, np.ones(ngal_cur[plane]),spec_z[gal_cur],kappa_pix_front*dz1 + kappa_pix_back*dz2 ])
        np.savetxt(outname, np.c_[ra, dec, shear1, shear2, w,z, den_pix, s11_pix, s22_pix, s12_pix])



    else:
        print('found no file named '+fname)


print("Done!")

starting analysis of plane 0
found no file named ../../GalCat/SRD-Y1/lin_bias_pos/GalCat_tomo1_plane0_0.6GpAM_RA_Dec_g1_g2_w_linear_bias_smooth0.5_bta2.asc
starting analysis of plane 1
found no file named ../../GalCat/SRD-Y1/lin_bias_pos/GalCat_tomo1_plane1_0.6GpAM_RA_Dec_g1_g2_w_linear_bias_smooth0.5_bta2.asc
starting analysis of plane 2
found no file named ../../GalCat/SRD-Y1/lin_bias_pos/GalCat_tomo1_plane2_0.6GpAM_RA_Dec_g1_g2_w_linear_bias_smooth0.5_bta2.asc
starting analysis of plane 3
found no file named ../../GalCat/SRD-Y1/lin_bias_pos/GalCat_tomo1_plane3_0.6GpAM_RA_Dec_g1_g2_w_linear_bias_smooth0.5_bta2.asc
starting analysis of plane 4
found no file named ../../GalCat/SRD-Y1/lin_bias_pos/GalCat_tomo1_plane4_0.6GpAM_RA_Dec_g1_g2_w_linear_bias_smooth0.5_bta2.asc
starting analysis of plane 5
found no file named ../../GalCat/SRD-Y1/lin_bias_pos/GalCat_tomo1_plane5_0.6GpAM_RA_Dec_g1_g2_w_linear_bias_smooth0.5_bta2.asc
starting analysis of plane 6
found no file named ../../GalCat/SR

In [11]:
# Concatenate catalogues:
import os

print('Concatenating SkySim5000 plane catalogues')
#V0:
#os.system("for tomo in {1..1}; do echo 'working on tomo'$tomo; ls -l   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/*tomo$tomo*0.6GpAM*; done")
#os.system("for tomo in {1..1}; do cat   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/*tomo$tomo*0.6GpAM* > /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/GalCat_tomo$tomo'_All_0.6GpAM_RA_Dec_g1_g2_w_z.asc'; done")
#os.system("for tomo in {1..5}; do echo 'working on tomo'$tomo; ls -l   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/SRD-Y1/random_pos/*tomo$tomo*0.6GpAM*_V2.asc; done")
#os.system("for tomo in {1..5}; do cat   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/SRD-Y1/random_pos/*tomo$tomo*0.6GpAM*_V2.asc > /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/SRD-Y1/random_pos/GalCat_tomo$tomo'_All_0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_smooth0.1Mpc.asc'; done")
#os.system("for tomo in {1..5}; do echo 'working on tomo'$tomo; ls -l   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/SRD-Y1/random_pos/*tomo$tomo*0.6GpAM*_smooth.asc; done")
#os.system("for tomo in {1..5}; do cat   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/SRD-Y1/random_pos/*tomo$tomo*0.6GpAM*_smooth.asc > /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/SRD-Y1/random_pos/GalCat_tomo$tomo'_All_0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_smooth0.5Mpc.asc'; done")

#V1:
#os.system("for tomo in {1..5}; do echo 'working on tomo'$tomo; ls -l   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/*tomo$tomo*0.06GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias.asc; done")
#os.system("for tomo in {1..5}; do cat   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/*tomo$tomo*0.06GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias.asc > /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/V1/GalCat_tomo$tomo'_All_0.06GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias.asc'; done")
#os.system("for tomo in {1..5}; do echo 'working on tomo'$tomo; ls -l   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/*tomo$tomo*0.06GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth.asc; done")
#os.system("for tomo in {1..5}; do cat   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/*tomo$tomo*0.06GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth.asc > /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/V1/GalCat_tomo$tomo'_All_0.06GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth.asc'; done")
#os.system("for tomo in {2..2}; do echo 'working on tomo'$tomo; ls -l   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/V1/*tomo$tomo*0.06GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth_1Mpc.asc; done")
#os.system("for tomo in {2..2}; do cat   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/V1/*tomo$tomo*0.06GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth_1Mpc.asc > /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/V1/GalCat_tomo$tomo'_All_0.06GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth_1Mpc.asc'; done")
#os.system("for tomo in {1..5}; do echo 'working on tomo'$tomo; rm -v   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/V1/*All_0.6GpAM*; done")
#os.system("for tomo in {1..5}; do echo 'working on tomo'$tomo; ls -l   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/V1/*tomo$tomo*0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth.asc; done")
#os.system("for tomo in {1..5}; do cat   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/V1/*tomo$tomo*0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth.asc > /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/V1/GalCat_tomo$tomo'_All_0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth.asc'; done")
#os.system("for tomo in {1..5}; do echo 'working on tomo'$tomo; ls -l   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/V1/*tomo$tomo*0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth_bta2.asc; done")
#os.system("for tomo in {1..5}; do cat   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/V1/*tomo$tomo*0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth_bta2.asc > /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/V1/GalCat_tomo$tomo'_All_0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth_bta2.asc'; done")
#os.system("let bta=2; for tomo in {1..5}; do echo 'working on tomo'$tomo; ls -l   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/SRD-Y1/lin_bias_pos/*tomo$tomo*0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth_bta$bta.asc; done")
#os.system("let bta=2; for tomo in {1..5}; do cat   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/SRD-Y1/lin_bias_pos/*tomo$tomo*0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth_bta$bta.asc > /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/SRD-Y1/lin_bias_pos/GalCat_tomo$tomo'_All_0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_bta'$bta'_smooth0.5Mpc.asc'; done")
#os.system("let bta=2; for tomo in {1..5}; do echo 'working on tomo'$tomo; ls -l   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/SRD-Y1/lin_bias_pos/*tomo$tomo*0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_V2_bta$bta.asc; done")
#os.system("let bta=2; for tomo in {1..5}; do cat   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/SRD-Y1/lin_bias_pos/*tomo$tomo*0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_V2_bta$bta.asc > /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/SRD-Y1/lin_bias_pos/GalCat_tomo$tomo'_All_0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_bta'$bta'_smooth0.1Mpc.asc'; done")
#os.system("let bta=1; smooth='1';for tomo in {1..5}; do echo 'working on tomo'$tomo; ls -l   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/SRD-Y1/lin_bias_pos/*tomo$tomo*0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth$smooth'__smooth_bta'$bta.asc; done")
#os.system("let bta=1; smooth='1';for tomo in {1..5}; do cat   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/SRD-Y1/lin_bias_pos/*tomo$tomo*0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth$smooth'__smooth_bta'$bta.asc > /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/SRD-Y1/lin_bias_pos/GalCat_tomo$tomo'_All_0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth'$smooth'_bta'$bta'_smooth0.5Mpc.asc'; done")
os.system("let bta=2; smooth='1';for tomo in {1..5}; do echo 'working on tomo'$tomo; ls -l   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/SRD-Y1/lin_bias_pos/*tomo$tomo*0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth$smooth'__smooth_bta'$bta.asc; done")
os.system("let bta=2; smooth='1';for tomo in {1..5}; do cat   /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/SRD-Y1/lin_bias_pos/*tomo$tomo*0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth$smooth'__smooth_bta'$bta.asc > /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/SRD-Y1/lin_bias_pos/GalCat_tomo$tomo'_All_0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth'$smooth'_bta'$bta'_smooth0.5Mpc.asc'; done")



Concatenating SkySim5000 plane catalogues
working on tomo1
-rw-rw---- 1 jharno lsst      2268 Oct 23 22:25 /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/SRD-Y1/lin_bias_pos/GalCat_tomo1_plane35_0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth1__smooth_bta2.asc
-rw-rw---- 1 jharno lsst     11879 Oct 23 22:25 /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/SRD-Y1/lin_bias_pos/GalCat_tomo1_plane36_0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth1__smooth_bta2.asc
-rw-rw---- 1 jharno lsst    124848 Oct 23 22:25 /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/SRD-Y1/lin_bias_pos/GalCat_tomo1_plane37_0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth1__smooth_bta2.asc
-rw-rw---- 1 jharno lsst    877344 Oct 23 22:25 /global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/SRD-Y1/lin_bias_pos/GalCat_tomo1_plane38_0.6GpAM_RA_Dec_g1_g2_w_z_den_Sij_linear_bias_smooth1__smooth_bta2.asc
-rw-rw---- 1 jharno lsst   4296509 Oct 23 22:25 /global/homes/j/jharno/IA-infusion/SkySim5000

0

In [ ]:
snaplist[plane]

In [ ]:
import matplotlib.pyplot as plt

plane = 56
Norm_empirical = 0.6525 

smooth_flag='_V2'
fname = "../../tidalfield/tidal_field_map_"+str(snaplist[plane])+"_allsky"+smooth_flag+".npy"

print("opening ", fname) 
hpmap_tidal = np.load(fname)
print("Got tidal file!")
#hpmap_tidal *= Norm_empirical

print("s11:",np.min(hpmap_tidal[:,0]), np.max(hpmap_tidal[:,0]), np.mean(hpmap_tidal[:,0]))
print("s22:",np.min(hpmap_tidal[:,1]), np.max(hpmap_tidal[:,1]), np.mean(hpmap_tidal[:,1]))
print("s12:",np.min(hpmap_tidal[:,2]), np.max(hpmap_tidal[:,2]), np.mean(hpmap_tidal[:,2]))

lonra = [0, 3]
latra = [0, 3]
hp.cartview(np.log(hpmap_tidal[:,0]), cbar=True, lonra=lonra, latra=latra,
        nest=False,
            title="plane "+np.str(plane),
            # min=-10, max=0,
            cmap=plt.cm.coolwarm,            
            norm=None, unit='$t_{11}$')
plt.show()

#--------------------
smooth_flag='_smooth'
fname = "../../tidalfield/tidal_field_map_"+str(snaplist[plane])+"_allsky"+smooth_flag+".npy"

print("opening ", fname) 
hpmap_tidal2 = np.load(fname)
print("Got tidal file!")
#hpmap_tidal *= Norm_empirical

print("s11:",np.min(hpmap_tidal2[:,0]), np.max(hpmap_tidal2[:,0]), np.mean(hpmap_tidal2[:,0]))
print("s22:",np.min(hpmap_tidal2[:,1]), np.max(hpmap_tidal2[:,1]), np.mean(hpmap_tidal2[:,1]))
print("s12:",np.min(hpmap_tidal2[:,2]), np.max(hpmap_tidal2[:,2]), np.mean(hpmap_tidal2[:,2]))

lonra = [0, 3]
latra = [0, 3]
hp.cartview(np.log(hpmap_tidal2[:,0]), cbar=True, lonra=lonra, latra=latra,
        nest=False,
            title="plane "+np.str(plane),
            # min=-10, max=0,
            cmap=plt.cm.coolwarm,            
            norm=None, unit='$t_{11}$')
plt.show()